초기화

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

print(load_dotenv(find_dotenv(), override=True))

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

True


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

from operator import itemgetter

In [3]:
# 프롬프트 분류기 

from typing import Literal
from langchain.pydantic_v1 import BaseModel
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

prompt 설정

In [4]:
general_prompt = PromptTemplate.from_template("""
너는 고객 문의를 매우 많이 해본 숙력된 종업원이야.
가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 친절하고 자세하게 답변해줘.
자연스럽게 주문으로 이어지도록 대화를 이어가되, 지나치게 주문을 유도하지는 말아줘.

가게에서 판매하는 상품 목록.
1. 상품: 떡케익5호
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 54,000원
2. 상품: 무지개 백설기 케익
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 51,500원
3. 상품: 미니 백설기
   기본 판매 수량: 35개
   기본 판매 수량의 가격: 31,500원
4. 상품: 개별 모듬팩
   기본 판매 수량: 1개
   기본 판매 수량의 가격: 13,500원
   
이전 대화 내용을 고려해서 답변해야 해.
이전 대화 내용은 다음과 같아:
{history}

고객이 문의는 다음과 같아:
{message}
답변:""")

order_change_prompt = PromptTemplate.from_template("""
너는 주문 변경을 전담하는 종업원이야.
고객이 변경한 주문 내용을 정확하게 파악하고, 너가 파악한 내용이 맞는지 고객에게 한 번 더 확인해줘.
너가 파악한 주문 변경 내용이 잘못됐다면, 주문 변경 내용을 정확히 파악하고 그 내용이 맞는지 고객에게 확인하는 작업을 주문 변경 내용을 정확히 파악할 때까지 반복해야돼.
고객의 주문 변경을 정확히 파악했다면, 고객에게 고객이 주문을 변경한 상품의 이름, 수량, 가격을 각각 알려주고, 마지막에는 변경된 주문의 총 가격을 알려줘.
이전 대화 내용을 고려해서 답변해야 해.

이전 대화 내용은 다음과 같아:
{history}


고객의 주문 변경은 다음과 같아:
{message}
답변:""")

order_cancel_prompt = PromptTemplate.from_template("""
너는 주문 취소를 전담하는 종업원이야.
고객이 취소하려는 주문을 정확하게 파악하고, 너가 파악한 내용이 맞는지 고객에게 한 번 더 확인해줘.
너가 파악한 주문 취소 내용이 잘못됐다면, 주문 취소 내용을 정확히 파악하고 그 내용이 맞는지 고객에게 확인하는 작업을 주문 취소 내용을 정확히 파악할 때
고객의 주문 취소 내용을 정확히 파악했다면, 고객에게 고객이 주문을 취소한 상품의 이름, 수량, 가격을 각각 알려주고, 마지막에는 취소된 주문의 총 가격을 알려줘.
이전 대화 내용을 고려해서 답변해야 해.

이전 대화 내용은 다음과 같아:
{history}

고객이 취소하려는 주문은 다음과 같아:
{message}
답변:""")

프롬프트 분류기 설정

In [5]:
class TopicClassifier(BaseModel):
    "사용자 문의의 주제를 분류해줘."
    
    topic: Literal["일반", "주문 변경", "주문 취소"]
    "사용자 문의의 주제는 '일반', '주문 변경', '주문 취소' 중 하나야."


classifier_function = convert_pydantic_to_openai_function(TopicClassifier)
llm = ChatOpenAI().bind(functions=[classifier_function], function_call={"name": "TopicClassifier"}) 
parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")
classifier_chain = llm | parser

prompt router 설정

In [6]:
prompt_branch = RunnableBranch(
  (lambda x: x["topic"] == "주문 변경", order_change_prompt),
  (lambda x: x["topic"] == "주문 취소", order_cancel_prompt),
  general_prompt
)

메모리 준비

In [7]:
memory = ConversationBufferMemory(return_messages=True)

입력 메시지를 받아 ai의 답변을 생성하는 chain 구성

In [8]:
chain = (
    RunnablePassthrough.assign(history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"))|
    RunnablePassthrough.assign(topic=itemgetter("message") | classifier_chain) 
    | prompt_branch 
    | ChatOpenAI()
    | StrOutputParser()
)


입력 메시지와 모델 출력 메시지(ai 답변)을 저장하는 함수 생성

In [9]:
def save_conversation(dict):
    print('customer_message: ', dict["customer_message"])
    print('ai_response: ', dict["ai_response"])
    memory.save_context({"inputs": dict["customer_message"]}, {"output": dict["ai_response"]})

최종 chain 구성

In [10]:
final_chain = {"customer_message": itemgetter("message"), "ai_response": chain} |  RunnableLambda(save_conversation)

동작 확인

In [11]:
final_chain.invoke({"message": "안녕하세요. 판매 중인 상품 정보 좀 알고 싶어요"})

customer_message:  안녕하세요. 판매 중인 상품 정보 좀 알고 싶어요
ai_response:  안녕하세요! 가게에서 판매 중인 상품 정보에 관심이 있으시군요. 저희 가게에서는 떡케익 5호, 무지개 백설기 케익, 미니 백설기, 그리고 개별 모듬팩을 판매하고 있습니다.

떡케익 5호는 1개를 기본 판매 수량으로 하고 있으며, 가격은 54,000원입니다. 무지개 백설기 케익도 1개를 기본 판매 수량으로 하고 있고, 가격은 51,500원입니다. 미니 백설기는 35개를 기본 판매 수량으로 하고 있으며, 가격은 31,500원입니다. 마지막으로 개별 모듬팩은 1개를 기본 판매 수량으로 하고 있고, 가격은 13,500원입니다.

어떤 상품에 대해 자세한 정보를 원하시나요? 저는 친절하게 답변해 드리겠습니다.


다른 구조 후보
- 문의 분류 chain
- general_query_chain, order_change_chain, order_cancel_chain 각각 구성
    - chain 구성: prompt | llm | RunnableLambda(save_conversation)
- RunnableBranch
-final_chain 구성: full_chain = {"topic": chain, "message": lambda x: x["message"]} | branch